In [1]:
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

/Users/dataly/opt/anaconda3/envs/pytorch_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
# Hyper parameter

EPOCHS = 400
LR = 1e-2
BS = 16384
SEED = 41

시드 고정

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PythonHashSeed'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

데이터 로드

In [4]:
train_df = pd.read_csv('./data/train.csv')
val_df = pd.read_csv('./data/val.csv')
test_df = pd.read_csv('./data/test.csv')

train_df = train_df.drop(columns = ['ID'])
val_df = val_df.drop(columns= ['ID'])

In [6]:
class MyDataset(Dataset):
    def _init_(self, df, eval_mode):
        self_df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns = ['Class']).values
        else:
            self.df = self.df.values
    
    def _getitem_(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def _len_(self):
        return(len(self.df))